## Making the dataset, thats is a load on a pod named "httpdia".


https://httpd.apache.org/docs/2.4/programs/ab.html

In [16]:
import time
from otumba.utils import manage_pods
from otumba.utils import get_pod_metrics_scenario_1
from GeneralPod import Pod

In [17]:
ServidorPrometheus = "http://192.168.49.2:30000/"

In [18]:
lapso="10m"
namespace="default"
timesleep = 1

In [19]:
respuesta = get_pod_metrics_scenario_1 (server = ServidorPrometheus, lapso = lapso, 
                                       namespace = namespace, podname = "httpdia", 
                                        requestpodname = "load-generator-jupyter-c")

In [20]:
print(respuesta)

{'pods': '5.0', 'file_descriptors': '14.5', 'receive_packets': '1456.4', 'transmit_packets': '1071.6', 'dropped_packets': '16.7', 'cpu_usage_seconds': '0.0764003339', 'cpu_throttled_seconds': '0.2494602084', 'memory_working_bytes': '3250762.4', 'memory_usage_bytes': '20.0'}


In [21]:
manage_pods(5, "httpd", "httpdia", "500m", "200m", "default")

Pod Exist, updating replicas


In [8]:
from datetime import datetime
import csv
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
header = list(respuesta.keys())
print(header)
print(respuesta)

['pods', 'file_descriptors', 'receive_packets', 'transmit_packets', 'dropped_packets', 'cpu_usage_seconds', 'cpu_throttled_seconds', 'memory_working_bytes', 'memory_usage_bytes', 'carga', 'fecha']
{'pods': '10.0', 'file_descriptors': '11.6', 'receive_packets': '734.0', 'transmit_packets': '539.5', 'dropped_packets': '6.7', 'cpu_usage_seconds': '0.0384423253', 'cpu_throttled_seconds': '0.0772288956', 'memory_working_bytes': '2510955.8', 'memory_usage_bytes': '20.0', 'carga': '1000', 'fecha': datetime.datetime(2021, 7, 16, 19, 49, 59, 12591)}


In [9]:
contador = 0
f = open("cargapod.csv", "w", newline='')
writer = csv.DictWriter(f, fieldnames = header)
writer.writeheader()
numberpods = 10
request = 50000
concurrency = 10000.0
host="http://httpdia.default.svc.cluster.local/"

In [10]:
loadpod = Pod(namepod = "load-generator-jupyter-c", 
              dockerimage = "httpd", namespace = "default", shell = "/bin/sh")

In [11]:
loadpod.create()

In [12]:
carga = "ab -n "+str(request)+" -c "+str(concurrency)+" "+host

In [13]:
loadpod.exec_command(carga)

ab -n 50000 -c 10000.0 http://httpdia.default.svc.cluster.local/


In [14]:
countstdout=0

In [15]:
while loadpod.resp.is_open():
    loadpod.resp.update(timeout=1)
    resp_stdout=""
    resp_stderr=""
    longitud =0
    #respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
    respuesta = get_pod_metrics_scenario_1 (server = ServidorPrometheus, lapso = lapso, 
                                       namespace = namespace, podname = "httpdia", 
                                        requestpodname = "load-generator-jupyter-c")
    respuesta["carga"] = str(concurrency)
    respuesta["fecha"] = datetime.now()
    print("Escritura:")
    print(respuesta)
    writer.writerow(respuesta)
    if loadpod.resp.peek_stdout():
        resp_stdout= loadpod.resp.read_stdout()
        longitud= len(resp_stdout)
        countstdout= countstdout+1
    if loadpod.resp.peek_stderr():
        resp_stderr=loadpod.resp.read_stderr()
        print("STDERR: %s" % resp_stderr)
        inicioerror = resp_stderr.find("Completed")
        if inicioerror == -1:
            failedrequest=resp_stderr
            break
    if  (longitud >300):
        #--------------------------
        inicio = resp_stdout.find("Failed requests")
        if (inicio > -1):
            fin = resp_stdout.find("Total transferred")
            failedrequest=resp_stdout[inicio+16:fin-1]
        finalresponse = resp_stdout.split("\n")
    if countstdout==2:
        break
respuesta =  failedrequest.strip()


Escritura:
{'pods': '10.0', 'file_descriptors': '11.6', 'receive_packets': '728.8', 'transmit_packets': '535.7', 'dropped_packets': '6.7', 'cpu_usage_seconds': '0.0381785227', 'cpu_throttled_seconds': '0.0768308521', 'memory_working_bytes': '2499042.3', 'memory_usage_bytes': '20.0', 'carga': '10000.0', 'fecha': datetime.datetime(2021, 7, 16, 19, 50, 3, 298266)}
STDERR: Completed 5000 requests

Escritura:
{'pods': '10.0', 'file_descriptors': '11.6', 'receive_packets': '728.6', 'transmit_packets': '535.6', 'dropped_packets': '6.7', 'cpu_usage_seconds': '0.0382321777', 'cpu_throttled_seconds': '0.0769253577', 'memory_working_bytes': '2500418.2', 'memory_usage_bytes': '20.0', 'carga': '10000.0', 'fecha': datetime.datetime(2021, 7, 16, 19, 50, 4, 188204)}
STDERR: Completed 10000 requests

Escritura:
{'pods': '10.0', 'file_descriptors': '11.6', 'receive_packets': '730.7', 'transmit_packets': '537.1', 'dropped_packets': '6.7', 'cpu_usage_seconds': '0.0382313946', 'cpu_throttled_seconds': '0.0

In [ ]:
while True:
    contador = contador +1
    concurrency = concurrency + 1000
    if concurrency > 20000:
        concurrency = 1000
        numberpods = numberpods + 1
    if numberpods > 20 :
        break
    
    manage_pods(numberpods, "httpd", "httpdia", "500m", "200m", "default")
        
    exec_command = ['/bin/sh']
    resp = stream(api_instance.connect_get_namespaced_pod_exec,
                  name,
                  'default',
                  command=exec_command,
                  stderr=True, stdin=True,
                  stdout=True, tty=False,
                  _preload_content=False)
    carga = "ab -n "+str(request)+" -c "+str(concurrency)+" "+host
    commands = [carga]       
    resp.write_stdin(carga + "\n")
    countstdout=0
    while resp.is_open():
        resp.update(timeout=1)
        resp_stdout=""
        resp_stderr=""
        longitud =0
        respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
        respuesta["carga"] = str(concurrency)
        respuesta["fecha"] = datetime.now()
        print("Escritura:")
        print(respuesta)
        writer.writerow(respuesta)
        if resp.peek_stdout():
            resp_stdout= resp.read_stdout()
            longitud= len(resp_stdout)
            countstdout= countstdout+1
        if resp.peek_stderr():
            resp_stderr=resp.read_stderr()
            print("STDERR: %s" % resp_stderr)
            inicioerror = resp_stderr.find("Completed")
            if inicioerror == -1:
                failedrequest=resp_stderr
                break
        if  (longitud >300):
            #--------------------------
            inicio = resp_stdout.find("Failed requests")
            if (inicio > -1):
                fin = resp_stdout.find("Total transferred")
                failedrequest=resp_stdout[inicio+16:fin-1]
            finalresponse = resp_stdout.split("\n")
        if countstdout==2:
            break
    respuesta =  failedrequest.strip()
   
    print (str(contador)+" Request: "+str(request) +" Concurrency: "+str(concurrency) +" Failed: " + respuesta)
    print (" Timesleep: "+str(timesleep))
    time.sleep(timesleep)
f.close()

In [ ]:
import csv
from datetime import datetime

In [ ]:
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
print(respuesta)

In [ ]:
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()

In [ ]:
header = list(respuesta.keys())
print(header)

In [ ]:
print(respuesta)

In [ ]:
with open("csv_write_dict.csv", "w", newline='') as f:
    
    writer = csv.DictWriter(f, fieldnames = header)
    writer.writeheader()
    writer.writerow(respuesta)

In [ ]:
f = open("csv_write_dict2.csv", "w", newline='')
writer = csv.DictWriter(f, fieldnames = header)
writer.writeheader()
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
respuesta= consulta_data(ServidorPrometheus,lapso,namespace)
respuesta["carga"] = '1000'
respuesta["fecha"] = datetime.now()
writer.writerow(respuesta)
f.close()

In [ ]:
from datetime import datetime

In [ ]:
now = datetime.now()

In [ ]:
print("now =", now)

In [ ]:
print("now =", now)

In [ ]:
type (now)

In [ ]:
with open("cargapod.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('cargapod.csv')

In [ ]:
datacarga = data.loc[data['carga'] == 5000]
datapodcarga = datacarga.loc[data['pods']== 1]

In [ ]:
print(datapodcarga.shape[0])

In [ ]:
print (datapodcarga)

In [ ]:
promedio =datapodcarga.mean()

In [ ]:
print(promedio)

In [ ]:
promedios = pd.DataFrame()
for i in range(20):
    for j in range (20):
        #print(str(i+1)+" "+ str((j+1)*1000))
        datacarga = data.loc[data['carga'] == ((j+1)*1000)]
        datapodcarga = datacarga.loc[data['pods']== (i+1)]
        promedio =datapodcarga.mean()
        promedios = promedios.append(promedio, ignore_index=True)       

In [ ]:
promedios.to_csv('cargapod_promedios.csv')

In [ ]:
maximos = data.max()
minimos = data.min()
print (maximos)
print (minimos)

In [ ]:
#ejemplo
df = pd.DataFrame({"Data1": [10, 20, 30], "Data2": [40, 50, 60]})
print(df)
column_maxes = df.max()
print(column_maxes)
df_max = column_maxes.max()
print(df_max)
normalized_df = df / df_max
print(normalized_df)

In [ ]:
column_maxes = data.max()
print(column_maxes)

In [ ]:
normalized_data = pd.DataFrame()
normalized_data['pods'] = data['pods'] 
normalized_data['file_descriptors'] = data['file_descriptors'] / column_maxes['file_descriptors']
normalized_data['receive_packets'] = data['receive_packets'] / column_maxes['receive_packets']
normalized_data['transmit_packets'] = data['transmit_packets'] / column_maxes['transmit_packets']
normalized_data['dropped_packets'] = data['dropped_packets'] / column_maxes['dropped_packets']
normalized_data['cpu_usage_seconds'] = data['cpu_usage_seconds'] / column_maxes['cpu_usage_seconds']
normalized_data['cpu_throttled_seconds'] = data['cpu_throttled_seconds'] / column_maxes['cpu_throttled_seconds']
normalized_data['memory_working_bytes'] = data['memory_working_bytes'] / column_maxes['memory_working_bytes']
normalized_data['memory_usage_bytes'] = data['memory_usage_bytes'] / column_maxes['memory_usage_bytes']
normalized_data['carga'] = data['carga']

In [ ]:
print(normalized_data)

In [ ]:
promedios = pd.DataFrame()
for i in range(20):
    for j in range (20):
        datacarga = normalized_data.loc[normalized_data['carga'] == ((j+1)*1000)]
        datapodcarga = datacarga.loc[normalized_data['pods']== (i+1)]
        promedio =datapodcarga.mean()
        promedios = promedios.append(promedio, ignore_index=True)       

In [ ]:
promedios.to_csv('cargapod_normalized_means.csv')